In [1]:
from gurobipy import *
from prettytable import PrettyTable
from math import pi
import numpy as np

In [2]:
OPF = Model('3 bus OPF')

In [3]:
#Sets
G = 3 # Generators
B = 3 # Bus
L = 3 #Lines
T = 2
Gen = range(G)
Bus = range(B)
Line = range(L)
Period = range(T)

### Input the susceptance for each transmission line

In [4]:
B1 = {(0,1):{'B':-10 ,'Limit':1000 },
      (0,2):{'B':-8 ,'Limit':1000 },
      (1,2):{'B':-5 ,'Limit':1500 } }

### Generate the graph

In [5]:
y = list(B1.keys())
x=list(map (lambda t: (t[1], t[0]), y))
B2=dict(zip(x,B1.values()))
B3 = {**B1, **B2}

In [6]:
tuple_list = list(B3.keys())
desired_list = [tuple(list(tup)+[t])  for t in Period for tup in tuple_list]
desired_list
vals=[B3[(i,j)] for (i,j) in B3.keys()]*T
B = dict(zip(desired_list,vals))
B

{(0, 1, 0): {'B': -10, 'Limit': 1000},
 (0, 2, 0): {'B': -8, 'Limit': 1000},
 (1, 2, 0): {'B': -5, 'Limit': 1500},
 (1, 0, 0): {'B': -10, 'Limit': 1000},
 (2, 0, 0): {'B': -8, 'Limit': 1000},
 (2, 1, 0): {'B': -5, 'Limit': 1500},
 (0, 1, 1): {'B': -10, 'Limit': 1000},
 (0, 2, 1): {'B': -8, 'Limit': 1000},
 (1, 2, 1): {'B': -5, 'Limit': 1500},
 (1, 0, 1): {'B': -10, 'Limit': 1000},
 (2, 0, 1): {'B': -8, 'Limit': 1000},
 (2, 1, 1): {'B': -5, 'Limit': 1500}}

In [7]:
L = {(0,0):200, (1,0):550, (2,0):100, (0,1):200, (1,1):550, (2,1):100} # electircal Load (MW)
Sbase = 100 # Power Base (MVA)

In [8]:
GenAt = {Gen[0]:{'a':0.001562 ,'b':7.92 ,'c':561, 'Pmin':150 ,'Pmax':600},
         Gen[1]:{'a':0.001940 ,'b':7.85 ,'c':310, 'Pmin':100 ,'Pmax':400},
         Gen[2]:{'a':0.004820 ,'b':7.97 ,'c':78,  'Pmin':50 , 'Pmax':200}}
GenAt

{0: {'a': 0.001562, 'b': 7.92, 'c': 561, 'Pmin': 150, 'Pmax': 600},
 1: {'a': 0.00194, 'b': 7.85, 'c': 310, 'Pmin': 100, 'Pmax': 400},
 2: {'a': 0.00482, 'b': 7.97, 'c': 78, 'Pmin': 50, 'Pmax': 200}}

In [9]:
Pmin = [(GenAt[i]['Pmin']) for i in Gen]
Pmax = [(GenAt[i]['Pmax']) for i in Gen]
g = [(a,t)  for t in Period for a in Gen]
b = [(a,t)  for t in Period for a in Bus] 

In [10]:
P = OPF.addVars(g, lb = Pmin*T, ub = Pmax*T, name = 'Pgen')
Delta = OPF.addVars(b, lb = -6*T, ub = 6*T, name = 'Delta')
Pij = OPF.addVars(B, lb = [-B[(i,j,t)]['Limit']  for (i,j,t) in B.keys()], 
                  ub = [B[(i,j,t)]['Limit'] for (i,j,t) in B.keys()], name = 'PF')
P

{(0, 0): <gurobi.Var Pgen[0,0]>,
 (1, 0): <gurobi.Var Pgen[1,0]>,
 (2, 0): <gurobi.Var Pgen[2,0]>,
 (0, 1): <gurobi.Var Pgen[0,1]>,
 (1, 1): <gurobi.Var Pgen[1,1]>,
 (2, 1): <gurobi.Var Pgen[2,1]>}

In [11]:
PowerFlow = OPF.addConstrs((Pij[(i,j,t)] == Sbase*(Delta[i,t]-Delta[j,t])*(B[(i,j,t)]['B']) for (i,j,t) in Pij.keys()), name = 'PowerFlow')
NetBus = OPF.addConstrs( (P[(i,t)] - (L[i,t])  == Pij.sum(i,'*',t) for (i,t) in g ), name = 'NetBus')

for t in Period:
    Delta[(0,t)].lb = 0
    Delta[(0,t)].ub = 0

In [12]:
OF = quicksum(GenAt[i]['c']+GenAt[i]['b']*P[(i,t)]+GenAt[i]['a']*P[(i,t)]*P[(i,t)] for (i,t) in g)

In [13]:
OPF.setObjective(OF, GRB.MINIMIZE)
OPF.Params.Method=-1
OPF.setParam( 'OutputFlag', True )
OPF.optimize()



status = OPF.status
if status == GRB.Status.INF_OR_UNBD or status == GRB.Status.INFEASIBLE \
or status == GRB.Status.UNBOUNDED:
    OPF.computeIIS()
    OPF.write("model.ilp")
OPF.write("model.lp")

Parameter Method unchanged
   Value: -1  Min: -1  Max: 5  Default: -1
Parameter OutputFlag unchanged
   Value: 1  Min: 0  Max: 1  Default: 1
Optimize a model with 18 rows, 24 columns and 54 nonzeros
Model has 6 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [8e+00, 8e+00]
  QObjective range [3e-03, 1e-02]
  Bounds range     [1e+01, 2e+03]
  RHS range        [1e+02, 6e+02]
Presolve removed 16 rows and 18 columns
Presolve time: 0.01s
Presolved: 2 rows, 6 columns, 6 nonzeros
Presolved model has 6 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 0.000e+00
 Factor NZ  : 2.000e+00
 Factor Ops : 2.000e+00 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   3.15802601e+06 -2.59102091e+06  2.50e+03 9.93e+02  1.01e+06     0s
   1   1.63988293e+04 -1.13528094e+05  2.50e-03 9.93e

In [14]:
OPF.printStats()


Statistics for model 3 bus OPF :
  Linear constraint matrix    : 18 Constrs, 24 Vars, 54 NZs
  Matrix coefficient range    : [ 1, 1000 ]
  Objective coefficient range : [ 7.85, 7.97 ]
  Variable bound range        : [ 12, 1500 ]
  RHS coefficient range       : [ 100, 550 ]


In [15]:
OPF.getVars()

[<gurobi.Var Pgen[0,0] (value 393.1698369628845)>,
 <gurobi.Var Pgen[1,0] (value 334.6037552894586)>,
 <gurobi.Var Pgen[2,0] (value 122.22640774765692)>,
 <gurobi.Var Pgen[0,1] (value 393.1698369628845)>,
 <gurobi.Var Pgen[1,1] (value 334.6037552894586)>,
 <gurobi.Var Pgen[2,1] (value 122.22640774765692)>,
 <gurobi.Var Delta[0,0] (value 0.0)>,
 <gurobi.Var Delta[1,0] (value 0.15817759661757372)>,
 <gurobi.Var Delta[2,0] (value 0.04374030043163849)>,
 <gurobi.Var Delta[0,1] (value 0.0)>,
 <gurobi.Var Delta[1,1] (value 0.15817759661757372)>,
 <gurobi.Var Delta[2,1] (value 0.04374030043163849)>,
 <gurobi.Var PF[0,1,0] (value 158.17759661757373)>,
 <gurobi.Var PF[0,2,0] (value 34.99224034531079)>,
 <gurobi.Var PF[1,2,0] (value -57.21864809296767)>,
 <gurobi.Var PF[1,0,0] (value -158.17759661757373)>,
 <gurobi.Var PF[2,0,0] (value -34.99224034531079)>,
 <gurobi.Var PF[2,1,0] (value 57.21864809296771)>,
 <gurobi.Var PF[0,1,1] (value 158.17759661757373)>,
 <gurobi.Var PF[0,2,1] (value 34.9922

In [16]:
Pij[(1,2)].lb

KeyError: (1, 2)

In [ ]:
tuple_list = [(0,1), (0,2), (1,2)]
time = range(5)

In [ ]:
desired_list = [tuple(list(tup)+[t])  for t in time for tup in tuple_list]

In [ ]:
desired_list